In [1]:
using LinearAlgebra
using Plots

In [2]:
fi = ComplexF32(1.0im)

0.0f0 + 1.0f0im

In [3]:
function vec2hermite(v::Vector{Float32})
    N = round(Int, sqrt(length(v)))
    H = zeros(ComplexF32, N, N)
    for i in 1:N
        for j in i:N
            l = N*(i-1) + 2j - i^2
            if(i == j)
                H[i,j] = v[l]
            else
                H[i,j] = v[l-1] + fi*v[l]
            end 
        end
    end
    H = Hermitian(H)
    return H
end

vec2hermite (generic function with 1 method)

In [4]:
function vec2unitary(v::Vector{Float32}, τ::Float32)
    H = vec2hermite(v)
    U = exp(fi*(τ*H))
    return U
end

vec2unitary (generic function with 1 method)

In [5]:
function make_unitary(N::Int, τ::Float32)
    v = rand(Float32, N^2)
    U = vec2unitary(v, τ)
    return U
end

make_unitary (generic function with 1 method)

In [6]:
function norm!(m::Hermitian{ComplexF32, Matrix{ComplexF32}})
    T = real(tr(m))
    m = m./T
end

norm! (generic function with 1 method)

In [7]:
function make_rand_dm(dim::Int)
    ρ_vec = rand(Float32, dim^2)
    rt_ρ = vec2hermite(ρ_vec)
    ρ = Hermitian(norm!(Hermitian(rt_ρ*rt_ρ')))
    return ρ
end

make_rand_dm (generic function with 1 method)

In [9]:
test_U = make_unitary(4, 1.0f0)
II = test_U * test_U'
println([real(II[i,i]) for i in 1:4])

Float32[0.9999999, 1.0000001, 0.99999994, 0.99999964]


In [10]:
function ehot(vs::Vector{ComplexF32}, i::Int, e_dim::Int)
    s_dim = length(vs)
    ve = zeros(ComplexF32, e_dim*s_dim)
    ve[(s_dim*(i-1)+1):(s_dim*i)] = vs
    #v2 = vcat(vs, ve)
    return ve
end

ehot (generic function with 1 method)

In [11]:
function make_ev(s_ev::Matrix{ComplexF32}, e_dim::Int)
    #e_vec::Vector{Matrix{Float32}} = []
    s_vec::Vector{Matrix{ComplexF32}} = []
    s_dim = size(s_ev)[1]
    tot_dim = s_dim * e_dim
    println(s_dim)
    for i in 1:s_dim
        sm = zeros(ComplexF32, tot_dim, e_dim)
        for j in 1:e_dim
            sm[:,j] = ehot(s_ev[:,i], j, e_dim)
            #push!(e_vec, twohot(i, j, s_dim))
            #push!(s_vec, ehot(s_ev[:,i], j, e_dim))
        end
        push!(s_vec, sm)
    end
    #=
    for j in 1:e_dim
        em = zeros(Float32, tot_dim, s_dim)
        for i in 1:s_dim
            em[:,i] = twohot(i, j, s_dim, e_dim)
        end
        push!(e_vec, em)
    end=#
    return s_vec
end

make_ev (generic function with 1 method)

In [13]:
function make_Mk(U::Matrix{ComplexF32}, s_vec::Vector{Matrix{ComplexF32}})
    L = size(U)[1]
    e_dim = length(s_vec)
    s_dim = div(L,e_dim)
    Ms::Vector{Matrix{ComplexF32}} = []
    for j in 1:e_dim
        for k in 1:s_dim
            push!(Ms, (s_vec[k]'*U*s_vec[j]))
        end
    end
    return Ms
end

make_Mk (generic function with 1 method)

In [25]:
#=
function make_Mk(U::Matrix{ComplexF32}, s_vec::Vector{Matrix{ComplexF32}}, e_vec::Vector{Matrix{Float32}})
    L = size(U)[1]
    e_dim = length(s_vec)
    s_dim = div(L,e_dim)
    Ms::Vector{Matrix{ComplexF32}} = []
    for j in 1:e_dim
        for k in 1:s_dim
            push!(Ms, (e_vec[k]'*U*s_vec[j]))
        end
    end
    return Ms
end=#

make_Mk (generic function with 1 method)

In [14]:
struct Dime
    s_dim::Int
    e_dim::Int
    tot_dim::Int
    M_size::Int
end

In [16]:
dim = Dime(2, 2, 4, 16)

Dime(2, 2, 4, 16)

In [17]:
function Λρ(ρ::Hermitian{ComplexF32, Matrix{ComplexF32}}, Ms::Vector{Matrix{ComplexF32}}, es::Vector{Float32}, ds::Dime)
    #ρ_vec = zeros(Float32, tot_dim^2)
    Lρ = zeros(ComplexF32, ds.s_dim, ds.s_dim)
    for i in 1:tot_dim
        Lρ += es[div(i-1,ds.e_dim)+1]*Ms[i]'*ρ*Ms[i]
    end
    return Hermitian(Lρ)
end

Λρ (generic function with 1 method)

In [18]:
testm = make_rand_dm(2)

2×2 Hermitian{ComplexF32, Matrix{ComplexF32}}:
 0.354515+0.0im       0.366608+0.129076im
 0.366608-0.129076im  0.645485+0.0im

In [19]:
function KL_divergence(ρ::Hermitian{ComplexF32, Matrix{ComplexF32}}, σ::Hermitian{ComplexF32, Matrix{ComplexF32}})
    return real(tr(ρ*(log(ρ)-log(σ))))
end

KL_divergence (generic function with 1 method)

In [20]:
struct DMs
    s_dm::Hermitian{ComplexF32, Matrix{ComplexF32}}
    e_dm::Hermitian{ComplexF32, Matrix{ComplexF32}}
    s_evs::Matrix{ComplexF32}
    s_es::Vector{Float32}

    U::Matrix{ComplexF32}
    Ms::Vector{Matrix{ComplexF32}}
end

function init_dms(ds::Dime, τ::Float32)
    s_dm = make_rand_dm(ds.s_dim)
    e_dm = make_rand_dm(ds.e_dim)
    s_es, s_evs = eigen(s_dm)
    U = make_unitary(ds.tot_dim, τ)
    s_evsa = make_ev(s_evs, ds.e_dim)
    Ms = make_Mk(U, s_evsa)
    return DMs(s_dm, e_dm, s_evs, s_es, U, Ms)
end

init_dms (generic function with 1 method)

In [25]:
function Λρ(ρ::Hermitian{ComplexF32, Matrix{ComplexF32}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float32, tot_dim^2)
    Lρ = zeros(ComplexF32, ds.s_dim, ds.s_dim)
    for i in 1:ds.tot_dim
        Lρ += dms.s_es[div(i-1,ds.e_dim)+1]*dms.Ms[i]*ρ*dms.Ms[i]'
    end
    return Hermitian(Lρ)
end

Λρ (generic function with 2 methods)

In [28]:
function Λρd(ρ::Hermitian{ComplexF32, Matrix{ComplexF32}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float32, tot_dim^2)
    Lρ = zeros(ComplexF32, ds.s_dim, ds.s_dim)
    for i in 1:ds.tot_dim
        Lρ += dms.s_es[div(i-1,ds.e_dim)+1]*dms.Ms[i]'*ρ*dms.Ms[i]
    end
    return Hermitian(Lρ)
end

Λρd (generic function with 1 method)

In [64]:
function Λρ2(ρ::Hermitian{ComplexF32, Matrix{ComplexF32}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float32, tot_dim^2)
    Lρ = zeros(ComplexF32, ds.s_dim, ds.s_dim)
    for i in 1:tot_dim
        Lρ += dms.s_es[((i-1)%ds.s_dim)+1]*dms.Ms[i]*ρ*dms.Ms[i]'
    end
    return Hermitian(Lρ)
end

Λρ2 (generic function with 1 method)

In [22]:
test_dms = init_dms(dim, 1.0f0)

2


DMs(ComplexF32[0.42669207f0 + 0.0f0im 0.2680267f0 + 0.28211877f0im; 0.2680267f0 - 0.28211877f0im 0.57330793f0 + 0.0f0im], ComplexF32[0.46014696f0 + 0.0f0im 0.22333215f0 + 0.31360644f0im; 0.22333215f0 - 0.31360644f0im 0.5398531f0 + 0.0f0im], ComplexF32[0.5302013f0 + 0.55807775f0im 0.43964547f0 + 0.46276078f0im; -0.6383069f0 + 0.0f0im 0.76978195f0 + 0.0f0im], Float32[0.10401594, 0.89598405], ComplexF32[0.313493f0 + 0.0048079835f0im -0.8688565f0 + 0.14714302f0im -0.15430166f0 - 0.10474976f0im 0.17983979f0 + 0.24085774f0im; 0.38208094f0 - 0.086137116f0im 0.14151652f0 + 0.01036604f0im -0.7126019f0 - 0.09704924f0im -0.55043125f0 - 0.079156f0im; 0.13280118f0 + 0.38294357f0im 0.008898728f0 + 0.41654897f0im 0.44536865f0 + 0.03738212f0im -0.5895205f0 + 0.33888137f0im; 0.15634486f0 + 0.7480427f0im 0.16708548f0 + 0.042344734f0im -0.31525353f0 + 0.38582787f0im 0.36823446f0 + 0.04932799f0im], Matrix{ComplexF32}[[0.32506344f0 - 0.45918882f0im -0.25576654f0 + 0.17491873f0im; 0.20893253f0 - 0.2024867f0

In [23]:
kl = KL_divergence(test_dms.s_dm, testm)

0.110944256f0

In [29]:
@show lp = Λρ(test_dms.s_dm, test_dms, dim)
@show tr(lp)

lp = Λρ(test_dms.s_dm, test_dms, dim) = ComplexF32[0.33888808f0 + 0.0f0im 0.1671971f0 + 0.23883107f0im; 0.1671971f0 - 0.23883107f0im 0.6611117f0 + 0.0f0im]
tr(lp) = 0.99999976f0


0.99999976f0

In [30]:
@show lpd = Λρd(test_dms.s_dm, test_dms, dim)
@show tr(lpd)

lpd = Λρd(test_dms.s_dm, test_dms, dim) = ComplexF32[0.64546895f0 + 0.0f0im 0.21882442f0 + 0.15587899f0im; 0.21882442f0 - 0.15587899f0im 0.46630204f0 + 0.0f0im]
tr(lpd) = 1.111771f0


1.111771f0